<a href="https://colab.research.google.com/github/soodeh-nilforoushan/NLP-Transformers/blob/main/POS_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

In [2]:
from datasets import load_dataset
import nltk
from nltk.corpus import brown
import json

import torch
from sklearn.metrics import f1_score,accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [4]:
corpus=brown.tagged_sents(tagset="universal")
corpus

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [5]:
# for sent in corpus:
#   brown_json=json.dumps(sent)
corpus_dict={}
words=[]
pos_tags=[]
# print(corpus[0][0][0])
for i,sentence in enumerate(corpus):
  word=[]
  pos_tag=[]
  for j,tuple in enumerate(corpus[i]):
    word.append(corpus[i][j][0])
    pos_tag.append(corpus[i][j][1])
  words.append(word)
  pos_tags.append(pos_tag)
print("words",words[0:2])
print("pos_tags",pos_tags[0:2])

import json
with open("data.json","w") as f:
  for x,y in zip(words,pos_tags):
    corpus_dict={'words':x, 'pos_tags':y}
    # print(corpus_dict)
    s=json.dumps(corpus_dict)
    f.write(f"{s}")

words [['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.']]
pos_tags [['DET', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', '.', 'DET', 'NOUN', '.', 'ADP', 'DET', 'NOUN', 'VERB', 'NOUN', '.'], ['DET', 'NOUN', 'ADV', 'VERB', 'ADP', 'NOUN', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'NOUN', '.', 'DET', 'VERB', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', '.', '.', 'VERB', 'DET', 'NOUN', 'CONJ', 'N

In [6]:
from datasets import load_dataset
data=load_dataset("json",data_files="data.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
#because dataset is too large and it takes time, we only use 20000 samples
dataset = data["train"].shuffle(seed=42).select(range(20_000))
split_data= dataset.train_test_split(test_size=0.3,seed=42)
split_data

DatasetDict({
    train: Dataset({
        features: ['words', 'pos_tags'],
        num_rows: 14000
    })
    test: Dataset({
        features: ['words', 'pos_tags'],
        num_rows: 6000
    })
})

In [8]:
from transformers import AutoTokenizer
#tokenize dataset
checkpoint="distilbert-base-cased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [9]:
data["train"][0]

{'words': ['The',
  'Fulton',
  'County',
  'Grand',
  'Jury',
  'said',
  'Friday',
  'an',
  'investigation',
  'of',
  "Atlanta's",
  'recent',
  'primary',
  'election',
  'produced',
  '``',
  'no',
  'evidence',
  "''",
  'that',
  'any',
  'irregularities',
  'took',
  'place',
  '.'],
 'pos_tags': ['DET',
  'NOUN',
  'NOUN',
  'ADJ',
  'NOUN',
  'VERB',
  'NOUN',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  'ADJ',
  'NOUN',
  'NOUN',
  'VERB',
  '.',
  'DET',
  'NOUN',
  '.',
  'ADP',
  'DET',
  'NOUN',
  'VERB',
  'NOUN',
  '.']}

In [10]:
t=tokenizer(data["train"][0]["words"],is_split_into_words=True)
t

{'input_ids': [101, 1109, 18196, 1391, 2224, 15169, 1163, 5286, 1126, 4449, 1104, 5161, 112, 188, 2793, 2425, 1728, 1666, 169, 169, 1185, 2554, 112, 112, 1115, 1251, 12692, 4233, 1261, 1282, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
t.tokens()

['[CLS]',
 'The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of',
 'Atlanta',
 "'",
 's',
 'recent',
 'primary',
 'election',
 'produced',
 '`',
 '`',
 'no',
 'evidence',
 "'",
 "'",
 'that',
 'any',
 'irregular',
 '##ities',
 'took',
 'place',
 '.',
 '[SEP]']

In [12]:
t.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 11,
 12,
 13,
 14,
 15,
 15,
 16,
 17,
 18,
 18,
 19,
 20,
 21,
 21,
 22,
 23,
 24,
 None]

In [13]:
#map targets to ints
#set is a uniqe list that why we use it
target=set()
for t in pos_tags:
  target=target.union(t)
target

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [14]:
target_list=list(target)
#order target
id2label={k:v for k,v in enumerate(target_list)}
label2id={v:k for k,v in id2label.items()}
print(label2id)
print(id2label)

{'VERB': 0, 'DET': 1, 'ADP': 2, 'X': 3, 'NUM': 4, 'ADV': 5, 'NOUN': 6, 'PRT': 7, 'CONJ': 8, 'ADJ': 9, 'PRON': 10, '.': 11}
{0: 'VERB', 1: 'DET', 2: 'ADP', 3: 'X', 4: 'NUM', 5: 'ADV', 6: 'NOUN', 7: 'PRT', 8: 'CONJ', 9: 'ADJ', 10: 'PRON', 11: '.'}


In [15]:
def align_target(labels,word_ids):
  #words id has the same length as tokenized input
  #labels is the original labels and has the same length as untokenized input
  aligned_labels=[] # it will be the same as tokenized input
  last_word=None
  for word in word_ids:
    if word is None:
      #it is a token like [CLS]
      label=-100 # this is the value the library uses to know to ignor
    else:
      #it is a real word
      #becaus NN can only work with numbers we convert it to integer thst we define above
      label=label2id[labels[word]]

    #add the label
    aligned_labels.append(label)

  return aligned_labels

In [16]:
data["train"].column_names

['words', 'pos_tags']

In [17]:
#it is not only tokenizing each input sentence in the batch but also it aligns the label
def tokenize_fn(batch):
  #tokenize the input sequence first
  #this populates input_ids, attention mask and etc.
  tokenized_inputs=tokenizer(
      batch["words"],truncation=True, is_split_into_words=True
  )

  labels_batch=batch['pos_tags'] #original targets
  aligned_labels_batch=[] #store align lables
  for i , labels in enumerate(labels_batch):
    word_ids=tokenized_inputs.word_ids(i)
    aligned_labels_batch.append(align_target(labels,word_ids))

  #the 'target' must be store in key called 'labels'
  tokenized_inputs['pos_tags'] =aligned_labels_batch

  return tokenized_inputs

In [18]:
tokenized_dataset=data.map(tokenize_fn, batched=True,remove_columns=data["train"].column_names)

Map:   0%|          | 0/57340 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['pos_tags', 'input_ids', 'attention_mask'],
        num_rows: 57340
    })
})

In [20]:
def flatten(list_of_lists):
  flattened=[val for sublist in list_of_lists for val in sublist]
  return flattened

In [21]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(logits_and_labels):
  logits,labels=logits_and_labels
  preds=np.argmax(logits,axis=-1)

  #remove -100 from labels and predictions
  labels_jagged=[[t for t in label if t!= -100] for label in labels]

  #do the same predictions whenever true label is -100
  preds_jagged=[[p for p,t in zip(ps,ts) if t!= -100] \
                for ps,ts in zip (preds,labels)
                ]
  #flatten labels and preds
  labels_flat = flatten(labels_jagged)
  preds_flat = flatten(preds_jagged)

  acc= accuracy_score(labels_flat,preds_flat)
  f1=f1_score(labels_flat,preds_flat,average="macro")

  return {
      "f1": f1,
      "accuracy": acc,
  }


In [22]:
labels=[[-100,0,0,1,2,1,-100]]
logits=np.array([[
    [0.8,0.1,0,1],
    [0.8,0.1,0,1],
    [0.8,0.1,0,1],
    [0.1,0.8,0,1],
    [0.1,0.8,0,1],
    [0.1,0.8,0,1],
    [0.1,0.8,0,1],
]])
compute_metrics((logits,labels))

{'f1': 0.0, 'accuracy': 0.0}

In [23]:
from transformers import AutoModelForTokenClassification
checkpoint="distilbert-base-cased"
model=AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [24]:
!pip install datasets transformers==4.28.0

In [25]:
from transformers import TrainingArguments

training_args=TrainingArguments(
    "distilbert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [29]:
from transformers import Trainer
#we are using data_collator, since the hugging face library does not use the correct version by default

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

KeyError: ignored

In [27]:
trainer.save_model("my_saved_model")

NameError: ignored

In [30]:
from transformers import pipeline
pipe=pipeline(
    "token-classification",
    model="my_saved_model",
    aggregation_startegy="simple",
    device=0,
)

OSError: ignored

In [ ]:
s="Bill gates was the CEO of Microsoft in Washangotn DC"
pipe(s)